In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error


In [2]:
df = pd.read_csv('./src/csv/melb_data.csv')
# df.columns.values
df.describe()

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,13580.000000,1.358000e+04,13580.000000,13580.000000,13580.000000,13580.000000,13518.000000,13580.000000,7130.000000,8205.000000,13580.000000,13580.000000,13580.000000
mean,2.937997,1.075684e+06,10.137776,3105.301915,2.914728,1.534242,1.610075,558.416127,151.967650,1964.684217,-37.809203,144.995216,7454.417378
std,0.955748,6.393107e+05,5.868725,90.676964,0.965921,0.691712,0.962634,3990.669241,541.014538,37.273762,0.079260,0.103916,4378.581772
min,1.000000,8.500000e+04,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1196.000000,-38.182550,144.431810,249.000000
25%,2.000000,6.500000e+05,6.100000,3044.000000,2.000000,1.000000,1.000000,177.000000,93.000000,1940.000000,-37.856822,144.929600,4380.000000
50%,3.000000,9.030000e+05,9.200000,3084.000000,3.000000,1.000000,2.000000,440.000000,126.000000,1970.000000,-37.802355,145.000100,6555.000000
75%,3.000000,1.330000e+06,13.000000,3148.000000,3.000000,2.000000,2.000000,651.000000,174.000000,1999.000000,-37.756400,145.058305,10331.000000
max,10.000000,9.000000e+06,48.100000,3977.000000,20.000000,8.000000,10.000000,433014.000000,44515.000000,2018.000000,-37.408530,145.526350,21650.000000


In [3]:
y = df["Price"]
df["YearBuilt"] = df["YearBuilt"].fillna(0)
feature_columns = ['Landsize', 'YearBuilt', 'Bedroom2', 'Bedroom2', 'Lattitude', 'Longtitude']
X = df[feature_columns]
X.head()

,Landsize,YearBuilt,Bedroom2,Bedroom2,Lattitude,Longtitude
0,202.0,0.0,2.0,2.0,-37.7996,144.9984
1,156.0,1900.0,2.0,2.0,-37.8079,144.9934
2,134.0,1900.0,3.0,3.0,-37.8093,144.9944
3,94.0,0.0,3.0,3.0,-37.7969,144.9969
4,120.0,2014.0,3.0,3.0,-37.8072,144.9941


A tree's depth is a measure of how many splits it makes before coming to a prediction.

In [4]:
# Specify Model
iowa_model = DecisionTreeRegressor()
# Fit model
iowa_model.fit(X,y)

print("First in-sample predictions:", iowa_model.predict(X.head()))
print("Actual target values for those homes:", y.head().tolist())

First in-sample predictions: [1480000. 1035000. 1465000.  850000. 1600000.]
Actual target values for those homes: [1480000.0, 1035000.0, 1465000.0, 850000.0, 1600000.0]


In [5]:
train_X, val_X, train_y, val_y = train_test_split(X,y, random_state=1)
dtr = DecisionTreeRegressor(random_state=1)
dtr.fit(train_X,train_y)
prediction = dtr.predict(val_X)
print("prediccion", prediction)
print("Actual:", val_y.head().tolist())

prediccion [1402000.  387000. 2300000. ... 1400000. 1200000.  650000.]
Actual: [1640000.0, 675000.0, 2800000.0, 615000.0, 2700000.0]


In [6]:
val_mse = mean_absolute_error(val_y, prediction)
print(val_mse)

244277.00186548848


# Underfitting and Overfitting

- The **max_leaf_nodes** argument provides a very sensible way to control overfitting vs underfitting.

In [7]:
def get_mae(max_leaf_node, train_X, val_X, train_y, val_y):
    dt = DecisionTreeRegressor(max_leaf_nodes=max_leaf_node, random_state=0)
    dt.fit(train_X, train_y)
    prediction = dt.predict(val_X)
    val_mae = mean_absolute_error(val_y, prediction)
    return(val_mae)

In [9]:
for max_leaf_nodes in [5, 50, 100, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

Max leaf nodes: 5  		 Mean Absolute Error:  359423
Max leaf nodes: 50  		 Mean Absolute Error:  262370
Max leaf nodes: 100  		 Mean Absolute Error:  250219
Max leaf nodes: 500  		 Mean Absolute Error:  225670
Max leaf nodes: 5000  		 Mean Absolute Error:  240711


Of the options listed, 500 is the optimal number of leaves.

**Overfitting:** capturing spurious patterns that won't recur in the future, leading to less accurate predictions, or
<br>
**Underfitting:** failing to capture relevant patterns, again leading to less accurate predictions.

In [13]:
dtr = DecisionTreeRegressor(max_leaf_nodes=500, random_state=0)
dtr.fit(X,y)

prediction_real = dtr.predict(val_X)
mae = mean_absolute_error(val_y, prediction_real)
print(f"mae: {mae}")

mae: 163902.08369090268
